### Import Activities File

In [ ]:
## Install python-benedict library
# !pip install python-benedict

## Install for Data Visualization
#!pip install ipyleaflet
#!jupyter nbextension enable --py --sys-prefix ipyleaflet 
#
# or just use conda
#!conda install -c conda-forge ipyleaflet

In [ ]:
import pandas as pd
from pathlib import Path
from os import listdir
from os.path import isfile, join
import gzip
import shutil
from benedict import benedict as bdict
from ipyleaflet import Map, Polyline


filepath = "data/"
fileOutputPath= filepath + "output/"

activityPath = filepath + "activities/"
activityOutputPath = fileOutputPath + "activities/"


def loadData():
    df = pd.read_csv(filepath+ 'activities.csv')
    data = df[df.columns[[0,1,2,3,4,5,6,10,13,14,15,16,17,18,19,20,21,22,23,26,27,31,32,41,44,45,47,50]]]

    rideTypes = ['Virtual Ride', 'Ride']
    rideData = data[data['Activity Type'].isin(rideTypes)]    
    return rideData.dropna(subset=['Filename']).reset_index()


def extractAllGPXFiles():
    # Create activity files Path if it's not exists
    Path(activityOutputPath).mkdir(parents=True, exist_ok=True)
    
    # check for all files in activities folder
    activityfiles = [f for f in listdir(filepath+"activities") if isfile(join(filepath+"activities", f))]

    # Extract all files from gz zip files
    for activity in activityfiles:
        # Extract activity filename
        filenamegz =activity
        filename=filenamegz[:-3]
       
        # Extract GPX File
        with gzip.open(activityPath + filenamegz, 'rb') as f_in:
            with open(activityOutputPath + filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    
    print("done!")
    
    
def getLocations(gpx_output_path,gpx_filename):
    # data-source can be an url, a filepath or data-string (as in this example)
    data_source = gpx_output_path + gpx_filename
    xmldata = bdict.from_xml(data_source)
    trk_list = xmldata['gpx.trk.trkseg.trkpt']
    
    locs = [[float(t['@lat']),float(t['@lon'])] for t in trk_list]
    
    return locs


def showOnMap(locations):

    line = Polyline(
        locations=locations,
        color="green" ,
        fill=False
    )

    m = Map(center = (float((min(locations)[0]+max(locations)[0])/2), float(min(locations)[1]+max(locations)[1])/2), zoom =12)
    m.add_layer(line)
    return m

data = loadData()
extractAllGPXFiles()

In [ ]:
len(data)

In [ ]:
fileindex = 125
showOnMap(getLocations(fileOutputPath,data['Filename'][fileindex][:-3]))

### Print Values

In [ ]:
data.columns

In [ ]:
def getDistance(dist):
    result = dist/1000
    return "{:.2f}".format(result)    

def getTimeInMinute(time):
    result = time/60
    return "{:.1f}".format(result)

In [ ]:
for index, row in data[-5:].iterrows():
    #print(fileOutputPath,row['Filename'][:-3])
    print("Name:", row['Activity Name'], end = "\t ")
    #print("Activity Date:", row['Activity Date'], end = "\t ")    
    print(f"Distance: { getDistance(row['Distance.1'])} km", end="\t ")
    print(f"Time: { getTimeInMinute(row['Elapsed Time'])} min")

    map = showOnMap(getLocations(fileOutputPath,row['Filename'][:-3]))

# Print the latest map        
map